In [1]:
import os
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PowerTransformer

In [4]:
df = pd.read_csv('../data/interim/Combined_Player_Per90_Cleaned.csv', index_col=0)

team_styles = pd.read_csv('../data/interim/cluster_assignments.csv', index_col=0)
prev_team_styles = team_styles.copy()
prev_team_styles['Season'] = prev_team_styles['Season'].str.split('-').map(lambda x: f'{int(x[0])+1}-{int(x[1])+1}')
prev_team_styles = prev_team_styles.rename(columns={'team_play_style': 'prev_season_squad_play_style'})

In [5]:
pd.set_option('display.max_columns', None) # reset when done viewing
df.head(10)

,PlayerID,Season,League,Player,Nation,Pos,Squad,Age,Born,90s,Gls_shoot,Sh_shoot,SoT_shoot,SoT%_shoot,Sh/90_shoot,SoT/90_shoot,G/Sh_shoot,G/SoT_shoot,Dist_shoot,FK_shoot,PK_shoot,PKatt_shoot,xG_shoot,npxG_shoot,npxG/Sh_shoot,G-xG_shoot,np:G-xG_shoot,SCA_gsc,SCA90_gsc,PassLive_gsc,PassDead_gsc,Drib_gsc,Sh_gsc,Fld_gsc,Def_gsc,GCA_gsc,GCA90_gsc,PassLive.1_gsc,PassDead.1_gsc,Drib.1_gsc,Sh.1_gsc,Fld.1_gsc,Def.1_gsc,Total Cmp_pass,Total Att_pass,Total Cmp%_pass,Total TotDist_pass,Total PrgDist_pass,Short Cmp_pass,Short Att_pass,Short Cmp%_pass,Medium Cmp_pass,Medium Att_pass,Medium Cmp%_pass,Long Cmp_pass,Long Att_pass,Long Cmp%_pass,Ast_pass,xA_pass,A-xA_pass,KP_pass,1/3_pass,PPA_pass,CrsPA_pass,Prog_pass,Pass Types: Live_pass_type,Pass Types: Dead_pass_type,Pass Types: FK_pass_type,Pass Types: TB_pass_type,Pass Types: Press_pass_type,Pass Types: Sw_pass_type,Pass Types: Crs_pass_type,Pass Types: CK_pass_type,Corner Kicks: In_pass_type,Corner Kicks: Out_pass_type,Corner Kicks: Str_pass_type,Height: Ground_pass_type,Height: Low_pass_type,Height: High_pass_type,Body Parts: Left_pass_type,Body Parts: Right_pass_type,Body Parts: Head_pass_type,Body Parts: TI_pass_type,Body Parts: Other_pass_type,Outcomes: Off_pass_type,Outcomes: Out_pass_type,Outcomes: Int_pass_type,Outcomes: Blocks_pass_type,Tackles: Tkl_def,Tackles: TklW_def,Tackles: Def 3rd_def,Tackles: Mid 3rd_def,Tackles: Att 3rd_def,Vs Dribbles: Tkl_def,Vs Dribbles: Att_def,Vs Dribbles: Tkl%_def,Vs Dribbles: Past_def,Pressures: Press_def,Pressures: Succ_def,Pressures: %_def,Pressures: Def 3rd_def,Pressures: Mid 3rd_def,Pressures: Att 3rd_def,Blocks: Blocks_def,Blocks: Sh_def,Blocks: ShSv_def,Blocks: Pass_def,Int_def,Tkl+Int_def,Clr_def,Err_def,Touches_poss,Def Pen_poss,Def 3rd_poss,Mid 3rd_poss,Att 3rd_poss,Att Pen_poss,Live_poss,Succ_poss,Att_poss,Succ%_poss,#Pl_poss,Megs_poss,Carries_poss,TotDist_poss,PrgDist_poss,Prog_poss,1/3_poss,CPA_poss,Mis_poss,Dis_poss,Targ_poss,Rec_poss,Rec%_poss,Prog.1_poss,Won_misc,Lost_misc,Won%_misc,MP_time,Min_time,Mn/MP_time,Min%_time,Starts_time,Mn/Start_time,Compl_time,Subs_time,Mn/Sub_time,unSub_time,PPM_time,onG_time,onGA_time,+/-_time,+/-90_time,On-Off_time,onxG_time,onxGA_time,xG+/-_time,xG+/-90_time,On-Off.1_time,prev_season_league
0,11975017,2017-2018,Serie A,Allan,BRA,MF,Napoli,26,1991,31.7,0.13,1.17,0.38,32.4,1.17,0.38,0.11,0.33,21.0,0.00,0.0,0.0,0.14,0.14,0.00,-0.01,-0.01,3.06,3.06,2.33,0.03,0.19,0.09,0.22,0.19,0.47,0.47,0.38,0.00,0.00,0.00,0.03,0.06,49.15,57.44,85.6,770.22,196.31,28.17,31.23,90.2,16.47,18.99,86.7,3.88,5.62,69.1,0.16,0.14,0.02,1.01,5.74,1.61,0.06,5.36,56.62,0.82,0.22,0.16,9.81,0.79,0.69,0.13,0.03,0.00,0.06,43.31,7.44,6.69,5.27,49.75,1.55,0.47,0.28,0.19,0.82,1.23,1.17,3.50,2.52,0.73,2.08,0.69,0.85,3.25,26.2,2.40,29.18,9.21,31.6,5.14,15.43,8.61,1.74,0.06,0.0,1.67,0.60,4.10,0.44,0.0,70.63,1.29,7.82,43.56,24.61,1.67,69.87,2.30,3.82,1.90,2.46,0.13,56.18,240.32,134.48,8.14,2.78,0.47,1.64,2.15,58.17,52.68,2.86,4.23,0.28,1.01,22.0,38,2850,75,83.3,32,0,19,6,0,0,2.39,62,21,41,1.29,0.19,57.0,20.2,36.8,1.16,0.47,0
1,11975017,2018-2019,Serie A,Allan,BRA,MF,Napoli,27,1991,29.1,0.03,0.86,0.07,8.0,0.86,0.07,0.04,0.50,23.6,0.00,0.0,0.0,0.04,0.04,0.00,-0.00,-0.00,2.99,2.99,2.47,0.00,0.24,0.10,0.10,0.07,0.31,0.31,0.27,0.00,0.00,0.03,0.00,0.00,68.08,76.80,88.6,1257.90,339.93,29.14,31.99,91.1,29.42,32.03,91.8,8.93,11.13,80.2,0.10,0.11,-0.01,1.24,9.18,1.27,0.03,6.80,73.13,3.68,2.37,0.41,9.31,1.48,0.31,0.03,0.00,0.00,0.00,56.49,10.55,9.76,5.22,68.93,1.65,0.58,0.31,0.45,0.72,1.31,1.48,4.26,3.16,1.55,2.10,0.62,1.44,4.43,32.6,2.99,32.27,9.07,28.1,8.87,18.42,4.98,2.27,0.31,0.0,1.96,0.55,4.81,0.86,0.0,88.76,2.75,18.93,57.73,17.84,1.17,85.53,1.92,2.68,2.47,1.99,0.07,71.00,322.78,179.97,8.18,2.06,0.45,1.03,1.31,68.66,64.81,3.24,2.20,0.27,0.55,33.3,33,2616,79,76.5,28,88,26,5,30,0,2.09,57,29,28,0.96,-0.16,51.6,26.4,25.2,0.87,-0.09,Serie A
2,11975017,2019-2020,Serie A,Allan,BRA,MF,Napoli,28,1991,14.2,0.14,0.85,0.49,58.3,0.84,0.49,0.17,0.29,20.0,0.00,0.0

In [6]:
target_cols = [
    'Gls_shoot',
    'G-xG_shoot',
    'SCA_gsc',
    'Ast_pass',
    'xA_pass',
    'Tackles: TklW_def',
    'Int_def',
    'Pressures: Succ_def',
    'Clr_def',
    'Won_misc',  # Aerial Duels won
]

targets_df = (
    df.loc[:, target_cols]
    .copy()
    .rename(columns={
        'Gls_shoot': 'Gls_tgt',
        'G-xG_shoot': 'G-xG_tgt',
        'SCA_gsc': 'SCA_tgt',
        'Ast_pass': 'Ast_tgt',
        'xA_pass': 'xA_tgt',
        'Tackles: TklW_def': 'TklW_tgt',
        'Int_def': 'Int_tgt',
        'Pressures: Succ_def': 'PressuresSucc_tgt',
        'Clr_def': 'Clr_tgt',
        'Won_misc': 'AerialDuelsWon_tgt',
    })
)

In [19]:
not_stats_cols = ['Season', 'League', 'Player', 'Nation', 'Pos', 'Squad', 'Age', 'Born', 'prev_season_league']

prev_season_df = (
    df.sort_values(['PlayerID', 'Season'])
    .groupby(['PlayerID'])
    .shift()
    .rename(columns={'Squad': 'prev_squad'})
    .drop([col for col in not_stats_cols if col != 'Squad'], axis=1)
)

In [29]:
prev_season_df

,prev_squad,90s,Gls_shoot,Sh_shoot,SoT_shoot,SoT%_shoot,Sh/90_shoot,SoT/90_shoot,G/Sh_shoot,G/SoT_shoot,Dist_shoot,FK_shoot,PK_shoot,PKatt_shoot,xG_shoot,npxG_shoot,npxG/Sh_shoot,G-xG_shoot,np:G-xG_shoot,SCA_gsc,SCA90_gsc,PassLive_gsc,PassDead_gsc,Drib_gsc,Sh_gsc,Fld_gsc,Def_gsc,GCA_gsc,GCA90_gsc,PassLive.1_gsc,PassDead.1_gsc,Drib.1_gsc,Sh.1_gsc,Fld.1_gsc,Def.1_gsc,Total Cmp_pass,Total Att_pass,Total Cmp%_pass,Total TotDist_pass,Total PrgDist_pass,Short Cmp_pass,Short Att_pass,Short Cmp%_pass,Medium Cmp_pass,Medium Att_pass,Medium Cmp%_pass,Long Cmp_pass,Long Att_pass,Long Cmp%_pass,Ast_pass,xA_pass,A-xA_pass,KP_pass,1/3_pass,PPA_pass,CrsPA_pass,Prog_pass,Pass Types: Live_pass_type,Pass Types: Dead_pass_type,Pass Types: FK_pass_type,Pass Types: TB_pass_type,Pass Types: Press_pass_type,Pass Types: Sw_pass_type,Pass Types: Crs_pass_type,Pass Types: CK_pass_type,Corner Kicks: In_pass_type,Corner Kicks: Out_pass_type,Corner Kicks: Str_pass_type,Height: Ground_pass_type,Height: Low_pass_type,Height: High_pass_type,Body Parts: Left_pass_type,Body Parts: Right_pass_type,Body Parts: Head_pass_type,Body Parts: TI_pass_type,Body Parts: Other_pass_type,Outcomes: Off_pass_type,Outcomes: Out_pass_type,Outcomes: Int_pass_type,Outcomes: Blocks_pass_type,Tackles: Tkl_def,Tackles: TklW_def,Tackles: Def 3rd_def,Tackles: Mid 3rd_def,Tackles: Att 3rd_def,Vs Dribbles: Tkl_def,Vs Dribbles: Att_def,Vs Dribbles: Tkl%_def,Vs Dribbles: Past_def,Pressures: Press_def,Pressures: Succ_def,Pressures: %_def,Pressures: Def 3rd_def,Pressures: Mid 3rd_def,Pressures: Att 3rd_def,Blocks: Blocks_def,Blocks: Sh_def,Blocks: ShSv_def,Blocks: Pass_def,Int_def,Tkl+Int_def,Clr_def,Err_def,Touches_poss,Def Pen_poss,Def 3rd_poss,Mid 3rd_poss,Att 3rd_poss,Att Pen_poss,Live_poss,Succ_poss,Att_poss,Succ%_poss,#Pl_poss,Megs_poss,Carries_poss,TotDist_poss,PrgDist_poss,Prog_poss,1/3_poss,CPA_poss,Mis_poss,Dis_poss,Targ_poss,Rec_poss,Rec%_poss,Prog.1_poss,Won_misc,Lost_misc,Won%_misc,MP_time,Min_time,Mn/MP_time,Min%_time,Starts_time,Mn/Start_time,Compl_time,Subs_time,Mn/Sub_time,unSub_time,PPM_time,onG_time,onGA_time,+/-_time,+/-90_time,On-Off_time,onxG_time,onxGA_time,xG+/-_time,xG+/-90_time,On-Off.1_time
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Napoli,31.7,0.13,1.17,0.38,32.4,1.17,0.38,0.11,0.33,21.0,0.00,0.00,0.00,0.14,0.14,0.00,-0.01,-0.01,3.06,3.06,2.33,0.03,0.19,0.09,0.22,0.19,0.47,0.47,0.38,0.00,0.00,0.00,0.03,0.06,49.15,57.44,85.6,770.22,196.31,28.17,31.23,90.2,16.47,18.99,86.7,3.88,5.62,69.1,0.16,0.14,0.02,1.01,5.74,1.61,0.06,5.36,56.62,0.82,0.22,0.16,9.81,0.79,0.69,0.13,0.03,0.00,0.06,43.31,7.44,6.69,5.27,49.75,1.55,0.47,0.28,0.19,0.82,1.23,1.17,3.50,2.52,0.73,2.08,0.69,0.85,3.25,26.2,2.40,29.18,9.21,31.6,5.14,15.43,8.61,1.74,0.06,0.00,1.67,0.60,4.10,0.44,0.0,70.63,1.29,7.82,43.56,24.61,1.67,69.87,2.30,3.82,1.90,2.46,0.13,56.18,240.32,134.48,8.14,2.78,0.47,1.64,2.15,58.17,52.68,2.86,4.23,0.28,1.01,22.0,38.0,2850.0,75.0,83.3,32.0,0.0,19.0,6.0,0.0,0.0,2.39,62.0,21.0,41.0,1.29,0.19,57.0,20.2,36.8,1.16,0.47
2,Napoli,29.1,0.03,0.86,0.07,8.0,0.86,0.07,0.04,0.50,23.6,0.00,0.00,0.00,0.04,0.04,0.00,-0.00,-0.00,2.99,2.99,2.47,0.00,0.24,0.10,0.10,0.07,0.31,0.31,0.27,0.00,0.00,0.03,0.00,0.00,68.08,76.80,88.6,1257.90,339.93,29.14,31.99,91.1,29.42,32.03,91.8,8.93,11.13,80.2,0.10,0.11,-0.01,1.24,9.18,1.27,0.03,6.80,73.13,3.68,2.37,0.41,9.31,1.48,0.31,0.03,0.00,0.00,0.00,56.49,10.55,9.76,5.22,68.93,1.65,0.58,0.31,0.45,0.72,1.31,

In [34]:
final_df = (
    df.loc[:, ['PlayerID'] + not_stats_cols]
    .join(prev_team_styles.set_index(['Squad', 'Season'])['prev_season_squad_play_style'].rename('squad_play_style'), on=['Squad', 'Season'])
    .join(prev_season_df)
    .join(prev_team_styles.set_index(['Squad', 'Season'])['prev_season_squad_play_style'].rename('prev_squad_play_style'), on=['prev_squad', 'Season'])
    .join(targets_df)
    .dropna()
    .drop(['Born', 'prev_squad'], axis=1)
)

final_df['squad_play_style'] = final_df['squad_play_style'].astype(int)
final_df['prev_squad_play_style'] = final_df['prev_squad_play_style'].astype(int)
final_df['Pos'] = final_df['Pos'].str.split(',').map(lambda x: x[0])  # Get primary position
final_df = final_df.loc[final_df['Pos'].ne('GK')].copy()  # Remove keepers
final_df = final_df.reset_index(drop=True)

# Reorder Columns
col_order = (
    ['PlayerID', 'Player', 'Nation', 'Season', 'Squad', 'prev_squad_play_style', 'squad_play_style', 'prev_season_league', 'League', 'Pos', 'Age']
    + final_df.drop('prev_squad_play_style', axis=1).columns[10:].tolist()
)

final_df = final_df.loc[:, col_order].copy()

final_df_model = final_df.loc[final_df['Season'].ne('2021-2022')].copy()
final_df_21_22 = final_df.loc[final_df['Season'].eq('2021-2022')].copy()

final_df_model.to_csv('../data/final/player-model-data.csv')
final_df_21_22.to_csv('../data/final/player-model-data_2021-2022.csv')

In [50]:
# Combine Prevous and Current Play Style & League columns

final_df_2 = final_df.copy()

final_df_2['prev_squad_to_squad_play_style'] = (
    final_df_2['prev_squad_play_style'].astype(str)
    + '-'
    + final_df_2['squad_play_style'].astype(str)
)

final_df_2['prev_league_to_league'] = (
    final_df_2['prev_season_league'].astype(str)
    + '-'
    + final_df_2['League'].astype(str)
)

col_order_2 = (
    final_df_2.columns[:5].tolist()
    + ['prev_squad_to_squad_play_style', 'prev_league_to_league']
    + final_df_2.columns[9:-2].tolist()
)

final_df_2 = final_df_2.loc[:, col_order_2].copy()

final_df_2_model = final_df_2.loc[final_df_2['Season'].ne('2021-2022')].copy()
final_df_2_21_22 = final_df_2.loc[final_df_2['Season'].eq('2021-2022')].copy()

final_df_2_model.to_csv('../data/final/player-model-data-2.csv')
final_df_2_21_22.to_csv('../data/final/player-model-data-2_2021-2022.csv')